In [1]:
# repo url(https://github.com/xwang71785/buffett.git)
# 只针对中国大陆股票市场
# 仅限于概念验证，勿用于实际投资中
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib as mpl

In [2]:
import pandas_datareader.data as web
import datetime as dt
import matplotlib.dates as mdates

In [3]:
# 构造数据源URL，根据需求变更stock中的股票代码来计算相对应的估值
server = "http://quotes.money.163.com/"
path = "service/"
table1 = "zcfzb_"
table2 = "lrb_"
table3 = "xjllb_"
stock = "600585"
type = ".html?type=year"

target1 = server + path + table1 + stock + type
target2 = server + path + table2 + stock + type
target3 = server + path + table3 + stock + type

In [4]:
# 获取三张财务报表原始数据
df1 = pd.read_csv(target1, header=0, prefix="V", encoding="gb18030")
df2 = pd.read_csv(target2, header=0, prefix="V", encoding="gb18030")
df3 = pd.read_csv(target3, header=0, prefix="V", encoding="gb18030")

In [6]:
# 报表行列转置
bs = df1.transpose()
pl = pd.DataFrame(df2.values.T, columns=df2['报告日期'], index=df2.columns)
cf = pd.DataFrame(df3.values.T, columns=df3[' 报告日期'], index=df2.columns)

In [7]:
bs.drop('报告日期', inplace=True)    # 删除第一行
bs.drop(bs.tail(1).index, inplace=True)    # 删除最后一行

In [8]:
# 将字符型数据转换成数值型
headers_bs = bs.columns
headers_pl = pl.columns
headers_cf = cf.columns


for head in headers_bs:
    bs[head] = pd.to_numeric(bs[head], errors='coerce').fillna(0) # 不符合转换条件的设为‘Nan’，再用0替换

for head in headers_pl:
    pl[head] = pd.to_numeric(pl[head], errors='coerce').fillna(0)

for head in headers_cf:
    cf[head] = pd.to_numeric(cf[head], errors='coerce').fillna(0)

In [9]:
bs_sum = pd.DataFrame()    # 利用column的数字标识

bs_sum['operat_current'] = bs[1]+bs[7]+bs[8]+bs[9]+bs[13]+bs[14]+bs[15]+bs[16]+bs[18]+bs[20]+bs[21]+bs[22]+bs[23]+bs[24]
bs_sum['operat_non_curr'] = bs[29]+bs[30]+bs[33]+bs[34]+bs[35]+bs[36]+bs[37]+bs[38]+bs[39]+bs[40]+bs[41]+bs[43]+bs[44]+bs[45]+bs[46]+bs[47]+bs[49]+bs[50]
bs_sum['operat_liability'] = bs[58]+bs[60]+bs[61]+bs[64]+bs[65]+bs[67]+bs[68]+bs[70]+bs[71]+bs[72]+bs[73]+bs[78]+bs[79]+bs[80]+bs[81]+bs[83]+bs[88]+bs[91]+bs[92]
bs_sum['financial_assets'] = bs[2]+bs[3]+bs[4]+bs[5]+bs[6]+bs[10]+bs[11]+bs[12]+bs[17]+bs[19]+bs[26]+bs[27]+bs[28]+bs[31]+bs[32]+bs[42]+bs[48]
bs_sum['financial_liability'] = bs[53]+bs[54]+bs[55]+bs[56]+bs[57]+bs[59]+bs[62]+bs[63]+bs[66]+bs[69]+bs[74]+bs[75]+bs[76]+bs[77]+bs[82]+bs[85]+bs[86]+bs[87]+bs[89]+bs[90]
bs_sum['equity'] = bs[95]+bs[96]+bs[98]+bs[99]+bs[100]+bs[102]+bs[104]+bs[106]
bs_sum['operat_net_assets'] = bs_sum['operat_current'] + bs_sum['operat_non_curr'] - bs_sum['operat_liability']
bs_sum['capital_expense'] = bs_sum['operat_net_assets'] - bs_sum['operat_net_assets'].shift(-1)

bs_sum = bs_sum.head(5)   # 只取过去5年数据
 

In [10]:
bs_sum 

,operat_current,operat_non_curr,operat_liability,financial_assets,financial_liability,equity,operat_net_assets,capital_expense
2021-12-31,12886999.0,32960204.0,5649535.0,16474384.0,1054712.0,38865450.0,40197668.0,3775620.0
2020-12-31,11707163.0,28754631.0,4039746.0,15422024.0,1094841.0,34327246.0,36422048.0,4179162.0
2019-12-31,10329869.0,26778877.0,4865860.0,13803860.0,1345522.0,29291910.0,32242886.0,2893520.0
2018-12-31,8571542.0,25472782.0,4694958.0,12215955.0,1201277.0,24234119.0,29349366.0,4056725.0
2017-12-31,5009246.0,24691691.0,4408296.0,11355227.0,1047870.0,19459638.0,25292641.0,1063984.0


In [11]:
pl_sum = pd.DataFrame()

pl_sum['total_revenu'] = pl['营业总收入(万元)']
pl_sum['growth_rate'] = (pl['营业总收入(万元)'] - pl['营业总收入(万元)'].shift(-1)) / pl['营业总收入(万元)']
pl_sum['intre_expen'] = pl['利息支出(万元)']
pl_sum['total_profit'] = pl['利润总额(万元)']
pl_sum['net_profit'] = pl['净利润(万元)']
pl_sum['income_tax'] = pl['所得税费用(万元)']
pl_sum['tax_rate'] = pl['所得税费用(万元)'] / pl['利润总额(万元)']

pl_sum.drop('报告日期', inplace=True)    # 删除第一行
pl_sum = pl_sum.head(5)

In [12]:
pl_sum

,total_revenu,growth_rate,intre_expen,total_profit,net_profit,income_tax,tax_rate
2021-12-31,16795266.0,-0.049359,0.0,4411620.0,3416584.0,995036.0,0.225549
2020-12-31,17624268.0,0.109011,0.0,4710792.0,3637018.0,1073774.0,0.227939
2019-12-31,15703033.0,0.182307,0.0,4455685.0,3435201.0,1020484.0,0.229030
2018-12-31,12840263.0,0.413479,0.0,3962920.0,3063601.0,899318.0,0.226933
2017-12-31,7531082.0,0.257319,0.0,2122876.0,1642873.0,480002.0,0.226109


In [13]:
cf_sum = pd.DataFrame()

cf_sum['depr_fixed'] = cf[' 固定资产折旧、油气资产折耗、生产性物资折旧(万元)']    # 注意column的标题有一个前置空格
cf_sum['intang_amort'] = cf[' 无形资产摊销(万元)']
cf_sum['deferred'] = cf[' 长期待摊费用摊销(万元)']
# cf_sum['depreciation'] = cf[' 固定资产折旧、油气资产折耗、生产性物资折旧(万元)']+cf[' 无形资产摊销(万元)']+cf[' 长期待摊费用摊销(万元)']

cf_sum.drop('报告日期', inplace=True)    # 删除第一行
cf_sum = cf_sum.head(5)

In [14]:
cf_sum

,depr_fixed,intang_amort,deferred
2021-12-31,535191.0,60485.0,0.0
2020-12-31,498444.0,44512.0,0.0
2019-12-31,493163.0,33934.0,0.0
2018-12-31,464730.0,30611.0,0.0
2017-12-31,459028.0,24290.0,0.0


In [15]:
valuation = pd.DataFrame()

valuation['profit_after_tax'] = pl_sum['net_profit'] + pl_sum['intre_expen'] * (1 - pl_sum['tax_rate'] )
valuation['depreciation'] = cf_sum['depr_fixed'] + cf_sum['intang_amort'] + cf_sum['deferred']
valuation['capital_expense'] = bs_sum['capital_expense']
valuation['net_cash_flow'] = valuation['profit_after_tax'] + valuation['depreciation'] - valuation['capital_expense']

In [16]:
valuation

,profit_after_tax,depreciation,capital_expense,net_cash_flow
2021-12-31,3416584.0,595676.0,3775620.0,236640.0
2020-12-31,3637018.0,542956.0,4179162.0,812.0
2019-12-31,3435201.0,527097.0,2893520.0,1068778.0
2018-12-31,3063601.0,495341.0,4056725.0,-497783.0
2017-12-31,1642873.0,483318.0,1063984.0,1062207.0


In [17]:
# 估算收入的增长率
growth_rate1 = pl_sum['growth_rate'].max()    # 最大
growth_rate2 = pl_sum['growth_rate'].mean()    # 平均
growth_rate3 = pl_sum['growth_rate'].min()    # 最小
# growth_rate3 = np.percentile(pl_sum['growth_rate'], 75)    # 75分位

discount_rate = 0.12    # 折现率，仅供参考

In [18]:
print(growth_rate1, growth_rate2, growth_rate3)

0.4134791475844381 0.18255133404128535 -0.04935926587884943


In [19]:
a = np.logspace(1,5,5, base=1+growth_rate1)
revenu_forecast = valuation.iloc[0, 0] * a
net_pre_value = np.npv(discount_rate, revenu_forecast)

In [20]:
revenu_forecast

array([ 4829270.23997063,  6826072.78224858,  9648511.53760206,
       13637969.86362837, 19276986.01762369])

In [21]:
net_pre_value

40573823.08708298

In [25]:
bs_rate = bs_sum.sum() / pl_sum['total_revenu'].sum()

In [54]:
bs_forecast = bs_rate.head(5) * revenu_forecast[0]    # 只提取前5项

b = np.logspace(0,4,5, base=1+growth_rate1)

np.expand_dims(b, 0).repeat(8, axis=0)
np.expand_dims(bs_forecast.T, 0).repeat(5, axis=1)

array([[3322880.97292518, 3322880.97292518, 3322880.97292518,
        3322880.97292518, 3322880.97292518, 9498945.75788108,
        9498945.75788108, 9498945.75788108, 9498945.75788108,
        9498945.75788108, 1620746.80858923, 1620746.80858923,
        1620746.80858923, 1620746.80858923, 1620746.80858923,
        4745524.01013883, 4745524.01013883, 4745524.01013883,
        4745524.01013883, 4745524.01013883,  393514.83793926,
         393514.83793926,  393514.83793926,  393514.83793926,
         393514.83793926]])

In [55]:
bs_forecast1 = bs_forecast * (1 + growth_rate1)
bs_forecast2 = bs_forecast1 * (1 + growth_rate1)
bs_forecast3 = bs_forecast2 * (1 + growth_rate1)
bs_forecast4 = bs_forecast3 * (1 + growth_rate1)
bs_forecast5 = bs_forecast4 * (1 + growth_rate1)

In [61]:
pd.concat([bs_last, bs_forecast, bs_forecast1, bs_forecast2, bs_forecast3, bs_forecast4], axis=1)

,2021-12-31,0,1,2,3,4
operat_current,12886999.0,3.322881e+06,4.696823e+06,6.638861e+06,9.383892e+06,1.326394e+07
operat_non_curr,32960204.0,9.498946e+06,1.342656e+07,1.897817e+07,2.682524e+07,3.791692e+07
operat_liability,5649535.0,1.620747e+06,2.290892e+06,3.238128e+06,4.577026e+06,6.469531e+06
financial_assets,16474384.0,4.745524e+06,6.707699e+06,9.481193e+06,1.340147e+07,1.894270e+07
financial_liability,1054712.0,3.935148e+05,5.562250e+05,7.862125e+05,1.111295e+06,1.570792e+06


In [60]:
bs_last = bs_sum.iloc[0].head(5)